# parsing the ['authorizedhighways'] column in the all_permits data for OSOW vehicle permits

## Duplicate created to clean the script while the other notebook works on pulling in the coordinates

- ns

- Originally requested by Stephen Yoon  
    - data provided by Stephen's office
- next step ==   
    - get the coordinates for the intersections

In [1]:
# import modules
import pandas as pd
import warnings
import gcsfs
import re

In [2]:
# the original_mapping is needed to standardize the highway names for the various records 
original_mapping = {
    "5": "I-5", "10": "I-10", "15": "I-15", "8": "I-8", "40": "I-40", "80": "I-80", "105": "I-105", "110": "I-110",
    "205": "I-205", "210": "I-210", "215": "I-215", "280": "I-280", "380": "I-380", "405": "I-405",
    "505": "I-505", "580": "I-580", "605": "I-605", "680": "I-680", "710": "I-710", "805": "I-805",
    "880": "I-880", "980": "I-980", "1": "SR-1", "2": "SR-2", "3": "SR-3", "4": "SR-4", "7": "SR-7",
    "9": "SR-9", "11": "SR-11", "12": "SR-12", "13": "SR-13", "14": "SR-14", "15": "SR-15",
    "16": "SR-16", "17": "SR-17", "18": "SR-18", "20": "SR-20", "22": "SR-22", "23": "SR-23",
    "24": "SR-24", "25": "SR-25", "26": "SR-26", "27": "SR-27", "28": "SR-28", "29": "SR-29",
    "32": "SR-32", "33": "SR-33", "34": "SR-34", "35": "SR-35", "36": "SR-36", "37": "SR-37",
    "38": "SR-38", "39": "SR-39", "41": "SR-41", "43": "SR-43", "44": "SR-44", "45": "SR-45",
    "46": "SR-46", "47": "SR-47", "49": "SR-49", "51": "SR-51", "52": "SR-52", "53": "SR-53",
    "54": "SR-54", "55": "SR-55", "56": "SR-56", "57": "SR-57", "58": "SR-58", "59": "SR-59",
    "60": "SR-60", "61": "SR-61", "62": "SR-62", "63": "SR-63", "65": "SR-65", "66": "SR-66",
    "67": "SR-67", "68": "SR-68", "70": "SR-70", "71": "SR-71", "72": "SR-72", "73": "SR-73",
    "74": "SR-74", "75": "SR-75", "76": "SR-76", "77": "SR-77", "78": "SR-78", "79": "SR-79",
    "82": "SR-82", "83": "SR-83", "84": "SR-84", "85": "SR-85", "86": "SR-86", "87": "SR-87",
    "88": "SR-88", "89": "SR-89", "90": "SR-90", "91": "SR-91", "92": "SR-92", "94": "SR-94",
    "96": "SR-96", "98": "SR-98", "imperial highway": "SR-99", "99": "SR-99", "103": "SR-103", "104": "SR-104", "107": "SR-107",
    "108": "SR-108", "109": "SR-109", "110": "SR-110", "111": "SR-111", "112": "SR-112",
    "113": "SR-113", "114": "SR-114", "115": "SR-115", "116": "SR-116", "118": "SR-118",
    "119": "SR-119", "120": "SR-120", "121": "SR-121", "123": "SR-123", "124": "SR-124",
    "125": "SR-125", "126": "SR-126", "127": "SR-127", "128": "SR-128", "129": "SR-129",
    "130": "SR-130", "131": "SR-131", "132": "SR-132", "133": "SR-133", "134": "SR-134",
    "135": "SR-135", "136": "SR-136", "137": "SR-137", "138": "SR-138", "139": "SR-139",
    "140": "SR-140", "142": "SR-142", "144": "SR-144", "145": "SR-145", "146": "SR-146",
    "147": "SR-147", "149": "SR-149", "150": "SR-150", "151": "SR-151", "152": "SR-152",
    "153": "SR-153", "154": "SR-154", "155": "SR-155", "156": "SR-156", "158": "SR-158",
    "160": "SR-160", "161": "SR-161", "162": "SR-162", "163": "SR-163", "164": "SR-164",
    "165": "SR-165", "166": "SR-166", "167": "SR-167", "168": "SR-168", "169": "SR-169",
    "170": "SR-170", "172": "SR-172", "173": "SR-173", "174": "SR-174", "175": "SR-175",
    "177": "SR-177", "178": "SR-178", "180": "SR-180", "182": "SR-182", "183": "SR-183",
    "184": "SR-184", "185": "SR-185", "186": "SR-186", "187": "SR-187", "188": "SR-188",
    "189": "SR-189", "190": "SR-190", "191": "SR-191", "192": "SR-192", "193": "SR-193",
    "197": "SR-197", "198": "SR-198", "200": "SR-200", "201": "SR-201", "202": "SR-202",
    "203": "SR-203", "204": "SR-204", "207": "SR-207", "210": "SR-210", "211": "SR-211",
    "213": "SR-213", "216": "SR-216", "217": "SR-217", "218": "SR-218", "219": "SR-219",
    "220": "SR-220", "221": "SR-221", "222": "SR-222", "223": "SR-223", "227": "SR-227",
    "229": "SR-229", "232": "SR-232", "233": "SR-233", "236": "SR-236", "237": "SR-237",
    "238": "SR-238", "241": "SR-241", "242": "SR-242", "243": "SR-243", "244": "SR-244",
    "245": "SR-245", "246": "SR-246", "247": "SR-247", "253": "SR-253", "254": "SR-254",
    "255": "SR-255", "259": "SR-259", "260": "SR-260", "261": "SR-261", "262": "SR-262",
    "263": "SR-263", "265": "SR-265", "266": "SR-266", "267": "SR-267", "269": "SR-269",
    "270": "SR-270", "271": "SR-271", "273": "SR-273", "275": "SR-275", "281": "SR-281",
    "282": "SR-282", "283": "SR-283", "284": "SR-284", "299": "SR-299", "330": "SR-330",
    "371": "SR-371", "780": "SR-780", "905": "SR-905", "6": "US-6", "50": "US-50",
    "95": "US-95", "97": "US-97", "101": "US-101", "199": "US-199", "395": "US-395"
}

# Generate extended mapping to include leading zeros
road_mapping = {}
for key, value in original_mapping.items():
    road_mapping[key] = value  # Original
    road_mapping[key.zfill(2)] = value  # 2-character zero-padded
    road_mapping[key.zfill(3)] = value  # 3-character zero-padded

#print(road_mapping)

In [3]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/freight/all_permits/"

In [4]:
file_names = ["all_permits_2023_sampleset.xlsx",
              "all_permits_2024_sampleset.xlsx"]

In [5]:
# Step 1
def load_excel_sheets_1(gcs_path, file_names):
    """
    Pull in the first sheet from each Excel file in GCS, add a 'year' column based on the filename,
    and remove records with NaN values in the 'permitnumber' column. Returns a concatenated DataFrame
    with data from all files.

    Parameters:
    gcs_path (str): The Google Cloud Storage path where the files are located.
    file_names (list): A list of Excel file names in the GCS path.

    Returns:
    pd.DataFrame: A single concatenated DataFrame with data from all files, a 'year' column, and
                  records with NaN values in 'permitnumber' removed.
    """
    
    # Create a Google Cloud Storage file system object
    fs = gcsfs.GCSFileSystem()
    
    # List to store all DataFrames
    df_list = []
    
    # Suppress any warnings
    warnings.filterwarnings("ignore")
    
    # Suppress the specific UserWarning
    warnings.filterwarnings(
        "ignore",
        message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project.",
        category=UserWarning,
        module="google.auth._default"
    )
    
    # Define the columns to keep
    columns_to_keep = ['permitnumber', 'year', 'permitvalidfrom', 'permitvalidto', 
                       'loaddescription', 'origin', 'destination', 'authorizedhighways']
    
    # Loop through each file in the file list
    for file in file_names:
        # Extract the year from the filename
        year = file.split('_')[2]  # Assuming the year is the third element when split by '_'
        
        # Open the file and read only the first sheet
        with fs.open(f"{gcs_path}{file}", 'rb') as f:
            df = pd.read_excel(f, sheet_name=0)  # Load only the first sheet
        
        # Clean headers by removing spaces and making characters lowercase
        df.columns = [col.replace(" ", "").lower() for col in df.columns]
        
        # Add 'year' column
        df['year'] = year
        
        # Filter columns and remove rows with NaN in 'permitnumber'
        df = df[columns_to_keep].dropna(subset=['permitnumber'])
        
        # Append to list
        df_list.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(df_list, ignore_index=True)
    
    return final_df

# Parsing function to create individual route locations
def parse_routes(route_info):
    segments = []

    # Split the data by "from", "to", or standalone dash patterns
    raw_segments = re.split(r'\s*-\s*from\s+|\s*-\s*to\s+|(?<!\s)-\s*', route_info)

    # Process each segment
    for segment in raw_segments:
        # Split by ' - ' or '-'
        sub_segments = re.split(r'\s*-\s*|\s*-\s*', segment)
        
        # Clean and add sub-segments
        segments.extend([sub.strip() for sub in sub_segments if sub.strip()])

    return segments


# Custom parsing function
def extract_location(text):
    # Stop keywords pattern
    stop_keywords = r"\b(?:dr|drive|rd|ave|way|pkwy|parkway|skyway|road|avenue|blvd|boulevard|st|street|line|lane|ln|hwy|highway)\b"

    # If 'from' exists, process it as before
    if "from" in text.lower():
        match = re.search(r"from\s+(`.*?`|'.*?'|\w+(?:\s+\w+)*)", text, re.IGNORECASE)
        if match:
            location = match.group(1)  # Extract the text after "from"
            # Keep the stop keywords and remove everything after them
            location = re.sub(r"(" + stop_keywords + r").*", r"\1", location, flags=re.IGNORECASE).strip()
            return location
    else:
        # If 'from' doesn't exist, look for a stop keyword and capture location
        match = re.search(r"(`.*?`|'.*?'|\w+(?:\s+\w+)*)\s+(" + stop_keywords + r")", text, re.IGNORECASE)
        if match:
            location = match.group(1)  # Capture location before stop keyword
            return location.strip()

    return None  # If no match is found


# Function to clean each string 
def clean_route(route):
    if not isinstance(route, str):  # Handle non-string entries
        return route
    
    # Define stop keywords regex (including 'exit')
    stop_keywords = r"\b(?:dr|drive|rd|ave|way|pkwy|parkway|skyway|road|avenue|blvd|boulevard|st|street|line|lane|ln|hwy|highway|exit)\b"

    # Extract numeric highway number with optional "rte" or "route" prefixes
    numeric_match = re.match(r"(?:rte|route)?\s*(\d+)", route, flags=re.IGNORECASE)
    if numeric_match:
        return numeric_match.group(1)  # Return the numeric value
    
    # Extract street name up to and including the first stop keyword (but excluding the stop word itself)
    street_match = re.search(rf"(.*?\b{stop_keywords}(?!\s*exit)\b)", route, flags=re.IGNORECASE)
    if street_match:
        return street_match.group(1).strip()  # Return the street name up to the stop keyword (excluding 'exit')
    
    # If neither is found, return the original string
    return route

# A function to parse the ['authorizedhighways'] column to get the route information
def process_route_locations(df, parse_routes, extract_location, clean_route, road_mapping):
    """
    Process a dataframe to format and clean route-related columns.
    
    Parameters:
    df (pandas.DataFrame): The input dataframe to process.
    parse_routes (function): Function to parse the 'authorizedhighways' column into route segments.
    extract_location (function): Function to extract location from a route segment.
    clean_route (function): Function to clean individual route location entries.
    road_mapping (dict): Mapping dictionary for road numbers to their corresponding classes.

    Returns:
    pandas.DataFrame: The processed dataframe with updated route locations and columns.
    """

    # Format the authorized highways field so the text is not all capitalized
    df['authorizedhighways'] = df['authorizedhighways'].str.capitalize()

    # Apply the parsing function to create lists of individual route locations
    df['route_segments'] = df['authorizedhighways'].apply(parse_routes)

    # Determine the maximum number of locations to create the necessary columns
    max_locations = df['route_segments'].apply(len).max()

    # Create new columns for each route location based on the maximum number of locations
    for i in range(max_locations):
        df[f'route_location_{i}'] = df['route_segments'].apply(lambda x: x[i] if i < len(x) else None)

    # Drop the temporary route_segments column
    df.drop(columns=['route_segments'], inplace=True)

    # Add a new column with all values set to "California"
    df.insert(5, "state", "California")  # Index 5 corresponds to the 6th column position

    # Apply title case to the 'origin' and 'destination' columns
    df['origin'] = df['origin'].str.title()
    df['destination'] = df['destination'].str.title()

    # Apply the extract_location function to the column
    df["route_location_start"] = df["route_location_0"].apply(extract_location)

    # Insert the route_location_start column into the 8th position
    df.insert(8, "route_location_start", df.pop("route_location_start"))

    # Drop the [authorizedhighways] column
    #df.drop(columns=['authorizedhighways'], inplace=True)

    # Drop the route_location_0 field
    df = df.drop(columns=['route_location_0'])

    # Identify target columns excluding "route_location_start"
    route_columns = [col for col in df.columns if col.startswith("route_location_") and col != "route_location_start"]

    # Apply the clean_route cleaning function to the target columns (columns that begin with the words "route_location")
    for col in route_columns:
        df[col] = df[col].apply(clean_route)

    # Iterate through each "route_location_" column to remove the word "exit"
    for col in route_columns:
        df[col] = df[col].apply(lambda x: str(x).replace("exit", "").strip() if isinstance(x, str) else x)

    for col in route_columns:
        df[col] = df[col].astype(str)

    # Update the road numbers to their corresponding road class numbers
    for col in route_columns:
        df[col] = df[col].astype(str).map(road_mapping).fillna(df[col])  # Keep original value if no mapping found

    # Create a new field called 'route_location_origin' that identifies the street and city/state
    df['route_location_origin'] = df['route_location_start'] + " " + df['origin'] + ", " + df['state']

    # Move the new column (['route_location_origin_0']) to the 9th position
    columns = list(df.columns)
    columns.insert(9, columns.pop(columns.index('route_location_origin')))
    df = df[columns]
    
    

    # Remove the 'route_location_destination_city' column if it exists
    if 'route_location_start' in df.columns:
        df = df.drop(columns=['route_location_start'])
    
    return df


# A function to create the ['route_intersection_x'] columns
def process_route_intersections(df):
    """
    Process a dataframe to identify and process route location and intersection columns.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe to process.
    
    Returns:
    pandas.DataFrame: A cleaned dataframe with processed route intersections.
    """

    # Identify all columns with "route_location_" prefix
    route_location_cols = [col for col in df.columns if col.startswith("route_location_")]

    # Initialize a counter for the new intersection column names
    intersection_counter = 0

    # Create new columns for intersections
    for i in range(len(route_location_cols) - 1):
        col1 = route_location_cols[i]
        col2 = route_location_cols[i + 1]

        # Name the new intersection column based on the counter
        intersection_col = f"route_intersection_{intersection_counter}"

        # Combine adjacent columns into one field (handle None gracefully)
        df[intersection_col] = df[col1].astype(str) + " and " + df[col2].astype(str)
        df[intersection_col] = df[intersection_col].replace("None and None", None)  # Optional cleanup for all-None rows
        intersection_counter += 1

    # Identify columns that start with "route_intersection_"
    intersection_cols = [col for col in df.columns if col.startswith("route_intersection_")]

    # Iterate over each intersection column
    for col in intersection_cols:
        # Replace values ending with " & None" with None (Null)
        df[col] = df[col].apply(lambda x: None if isinstance(x, str) and x.endswith(" and None") else x)

    # Identify columns that start with "route_intersection_"
    intersection_cols = [col for col in df.columns if col.startswith("route_intersection_")]

    # Iterate over each intersection column to remove leading zeros from numeric values
    for col in intersection_cols:
        # Apply the transformation to each value in the column
        df[col] = df[col].apply(lambda x: ' and '.join([part.lstrip('0') if part.isdigit() else part for part in str(x).split(' and ')]) if isinstance(x, str) else x)

    # Create a list(?) called core_columns to be included in the next iteration of the dataframe
    core_columns = [
        "permitnumber", "year", "permitvalidfrom", "permitvalidto",
        "loaddescription", "state", "origin", "destination", "authorizedhighways", "route_location_origin"
    ]

    # subset_columns combines the core columns with the intersection_cols identified earlier in the script
    subset_columns = core_columns + intersection_cols

    # this next line utilizes the defined subset_columns to create a cleaned up version of the dataframe including only the columns needed for this analysis 
    df = df[subset_columns]

    # Replace None values in intersection_cols with empty strings
    for col in intersection_cols:
        df[col] = df[col].apply(lambda x: "" if x is None else x)
    
    return df

# Function to get the last 'route_intersection_x' field
def get_last_intersection(row):
    # Identify columns that match the pattern 'route_intersection_x'
    intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]
    # Get the last non-null value among these columns
    return row[intersection_columns].dropna().iloc[-1] if intersection_columns else None

def add_route_location_destination_city(df):
    # Function to get the last 'route_intersection_x' value
    def get_last_intersection(row):
        # Identify columns that match the pattern 'route_intersection_x'
        intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]
        # Get the last non-null value among these columns
        return row[intersection_columns].dropna().iloc[-1] if len(intersection_columns) > 0 else None

    # Create the new column
    df['route_location_destination_city'] = df.apply(
        lambda row: f"{get_last_intersection(row)} {row['destination']}, {row['state']}", axis=1
    )

    return df

def create_route_intersection_last(df):
    """
    Create a 'route_intersection_last' column to capture the last non-null value
    from all 'route_intersection_x' columns.

    Parameters:
    df (pandas.DataFrame): The dataframe to process.

    Returns:
    pandas.DataFrame: The dataframe with the new 'route_intersection_last' column.
    """
    # Identify all 'route_intersection_x' columns
    intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]

    if not intersection_columns:
        raise ValueError("No 'route_intersection_' columns found in the dataframe.")

    # Ensure the columns are processed in order
    intersection_columns = sorted(intersection_columns, key=lambda x: int(x.split('_')[-1]))

    # Create 'route_intersection_last' by finding the last non-null value row-wise
    df['route_intersection_last'] = df[intersection_columns].apply(
        lambda row: next((val for val in reversed(row) if pd.notnull(val) and val != ''), None), axis=1
    )

    return df

def create_route_intersection_destination(df):
    """
    Create a new column 'route_intersection_destination' by combining 'route_intersection_last' 
    and 'route_location_destination_city'. Then, clean up the column by removing extra whitespace.
    
    Parameters:
    df (pandas.DataFrame): The input dataframe to process.
    
    Returns:
    pandas.DataFrame: The dataframe with the new 'route_intersection_destination' column and cleaned columns.
    """
    # Create the new 'route_intersection_destination' column
    df['route_intersection_destination'] = df['route_intersection_last'] + " " + df['route_location_destination_city']
    
    # Remove any extra whitespace or empty strings in the resulting combination
    df['route_intersection_destination'] = df['route_intersection_destination'].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # Drop the 'route_location_destination_city' column if it exists
    if 'route_location_destination_city' in df.columns:
        df.drop(columns=['route_location_destination_city'], inplace=True)

    # Drop the 'route_intersection_last' column if it exists
    if 'route_intersection_last' in df.columns:
        df.drop(columns=['route_intersection_last'], inplace=True)

    return df

# A function to clean "route_intersection" columns
def clean_route_intersections(df):
    # Find all columns starting with "route_intersection_"
    intersection_columns = [col for col in df.columns if col.startswith("route_intersection_")]
    
    # Replace variations of "imperial highway" (e.g., "imperial hwy") with "SR-99"
    for col in intersection_columns:
        df[col] = df[col].str.replace(
            r"(?i)\bimperial (highway|hwy)\b", "SR-99", regex=True
        )  # Matches "imperial highway" or "imperial hwy"
    
    return df

def process_workflow(gcs_path, file_names):
    # Load the Excel sheets from GCS and get the initial DataFrame
    df = load_excel_sheets_1(gcs_path, file_names)
    
    # Process route locations
    df = process_route_locations(df, parse_routes, extract_location, clean_route, road_mapping)
       
    # Process route intersections
    df = process_route_intersections(df)
    
    # Add route location destination city
    df = add_route_location_destination_city(df)
    
    # Create route intersection last
    df = create_route_intersection_last(df)
    
    # Create route intersection destination
    df = create_route_intersection_destination(df)

    # this is a new script - intended to help clean up the ["route_locations_x"] before they become intersections
    df = clean_route_intersections(df)
    
    
    # # Replace the last non-null route intersection with destination
    # df = replace_last_non_null_intersection(df)
    
    return df

In [6]:
df = process_workflow(gcs_path, file_names)

/opt/conda/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
df.to_csv("justlooking_subset_v1_od.csv", index=False)

In [8]:
# Find all columns starting with "route_intersection_"
intersection_columns = [col for col in df.columns if col.startswith("route_intersection_")]


def normalize_intersection(intersection_columns):
    """
    Normalizes intersections by ordering highway identifiers numerically.
    If both sides of the intersection are highways (I-, SR-, or US-),
    ensures the smaller-numbered highway appears first.

    Args:
        intersection (str): The intersection string in the format "Location1 and Location2".

    Returns:
        str: The normalized intersection string, or the original string if no changes are needed.
    """

    
    
    if not intersection_columns or pd.isna(intersection_columns):
        return intersection_columns

    parts = [part.strip() for part in intersection_columns.split("and")]
    if len(parts) != 2:
        return intersection_columns  # Return as-is if not exactly two parts

    pattern = r"^(I-|SR-|US-)(\d+)$"  # Pattern to match highway identifiers

    match1 = re.match(pattern, parts[0])
    match2 = re.match(pattern, parts[1])

    if match1 and match2:
        # Extract numeric portions and compare
        num1 = int(match1.group(2))
        num2 = int(match2.group(2))

        if num1 > num2:
            # Swap to ensure the smaller number comes first
            parts = [parts[1], parts[0]]

    return " and ".join(parts)


def process_intersections(df, intersection_columns):
    """
    Processes and normalizes intersections in specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to process.

    Returns:
        pd.DataFrame: The updated DataFrame with normalized intersections.
    """
    for col in intersection_columns:
        if col in df.columns:
            df[col] = df[col].apply(normalize_intersection)
    return df

In [9]:
df = process_intersections(df, intersection_columns)

In [10]:
df.head()

,permitnumber,year,permitvalidfrom,permitvalidto,loaddescription,state,origin,destination,authorizedhighways,route_location_origin,...,route_intersection_15,route_intersection_16,route_intersection_17,route_intersection_18,route_intersection_19,route_intersection_20,route_intersection_21,route_intersection_22,route_intersection_23,route_intersection_destination
0,e23-013125,2023,02/15/2023,02/21/2023,75' KELLY BAR,California,Hayward,Antelope,* from clawiter rd s/b on ramp - 092e - 880n -...,"clawiter rd Hayward, California",...,I-80 and antelope rd,,,,,,,,,"I-80 and antelope rd Antelope, California"
1,e23-021610,2023,03/20/2023,03/26/2023,UNLADEN 9 AXLE WITH 2 DECK INSERTS,California,Fontana,Ontario,* from sierra ave w/b on ramp - 015s - 060w - ...,"sierra ave Fontana, California",...,,,,,,,,,,"SR-60 and vineyard ave Ontario, California"
2,e23-022752,2023,03/22/2023,03/28/2023,M95 TRACKED CONVEYOR,California,Dixon,Fresno,* from industrial way - 113n - 080w - 680s - 5...,"industrial way Dixon, California",...,,,,,,,,,,"SR-180 and north floyd ave Fresno, California"
3,e23-036568,2023,05/05/2023,05/11/2023,5 TROWEL MACHINES (END TO END) & MISC LEGAL FR...,California,Elk Grove,Ca/Nv Border,* from grant line rd w/b on ramp - 099n - 051n...,"grant line Elk Grove, California",...,,,,,,,,,,"I-80 and nv line Ca/Nv Border, California"
4,e23-017248,2023,03/02/2023,03/08/2023,150H GRADER,California,Fairfield,Saratoga,* from air base pkwy n/b on ramp - 080w - 680s...,"air base pkwy Fairfield, California",...,,,,,,,,,,"SR-17 and scotts valley dr Saratoga, California"


In [11]:
df.to_csv("justlooking_subset_v2.csv", index=False)

In [12]:
def clean_intersections_1(df):
    """
    Cleans the 'route_intersection_0' column in the DataFrame by replacing specific values.
    
    - Replaces values that start with "az line" and end with "I-40" with 
      "colorado river bridge and I-10, California".
    
    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The updated DataFrame with cleaned intersections.
    """
    # Check if the column exists
    if "route_intersection_0" in df.columns:
        # Debugging: Display the initial state of the column
        print("Before cleaning:")
        print(df["route_intersection_0"].head())
        
        # Define the replacement logic
        def replace_intersection(value):
            if isinstance(value, str) and value.lower().startswith("az line") and value.endswith("I-40"):
                return "colorado river bridge and I-10, California"
            return value

        # Apply the function to clean the column
        df["route_intersection_0"] = df["route_intersection_0"].apply(replace_intersection)
        
        # Debugging: Display the updated state of the column
        print("After cleaning:")
        print(df["route_intersection_0"].head())
    else:
        print("Column 'route_intersection_0' does not exist in the DataFrame.")

    return df

In [13]:
df = clean_intersections_1(df)

Before cleaning:
0       clawiter rd Hayward, California and SR-92
1        sierra ave Fontana, California and SR-15
2     industrial way Dixon, California and SR-113
3      grant line Elk Grove, California and SR-99
4    air base pkwy Fairfield, California and I-80
Name: route_intersection_0, dtype: object
After cleaning:
0       clawiter rd Hayward, California and SR-92
1        sierra ave Fontana, California and SR-15
2     industrial way Dixon, California and SR-113
3      grant line Elk Grove, California and SR-99
4    air base pkwy Fairfield, California and I-80
Name: route_intersection_0, dtype: object


In [14]:
df.head()

,permitnumber,year,permitvalidfrom,permitvalidto,loaddescription,state,origin,destination,authorizedhighways,route_location_origin,...,route_intersection_15,route_intersection_16,route_intersection_17,route_intersection_18,route_intersection_19,route_intersection_20,route_intersection_21,route_intersection_22,route_intersection_23,route_intersection_destination
0,e23-013125,2023,02/15/2023,02/21/2023,75' KELLY BAR,California,Hayward,Antelope,* from clawiter rd s/b on ramp - 092e - 880n -...,"clawiter rd Hayward, California",...,I-80 and antelope rd,,,,,,,,,"I-80 and antelope rd Antelope, California"
1,e23-021610,2023,03/20/2023,03/26/2023,UNLADEN 9 AXLE WITH 2 DECK INSERTS,California,Fontana,Ontario,* from sierra ave w/b on ramp - 015s - 060w - ...,"sierra ave Fontana, California",...,,,,,,,,,,"SR-60 and vineyard ave Ontario, California"
2,e23-022752,2023,03/22/2023,03/28/2023,M95 TRACKED CONVEYOR,California,Dixon,Fresno,* from industrial way - 113n - 080w - 680s - 5...,"industrial way Dixon, California",...,,,,,,,,,,"SR-180 and north floyd ave Fresno, California"
3,e23-036568,2023,05/05/2023,05/11/2023,5 TROWEL MACHINES (END TO END) & MISC LEGAL FR...,California,Elk Grove,Ca/Nv Border,* from grant line rd w/b on ramp - 099n - 051n...,"grant line Elk Grove, California",...,,,,,,,,,,"I-80 and nv line Ca/Nv Border, California"
4,e23-017248,2023,03/02/2023,03/08/2023,150H GRADER,California,Fairfield,Saratoga,* from air base pkwy n/b on ramp - 080w - 680s...,"air base pkwy Fairfield, California",...,,,,,,,,,,"SR-17 and scotts valley dr Saratoga, California"


In [15]:
df.to_csv("justlooking_subset_v4.csv", index=False)

In [16]:
def update_route_intersection(df):
    # Create a new column with the same values as ['route_intersection_0']
    df['route_intersection_origin'] = df['route_intersection_0']
    
    # Replace the specific string
    df.loc[df['route_intersection_0'] == "colorado river bridge and I-10, California", 
           'route_intersection_origin'] = "I-10 and Arizona Line"
    
    # Place the new column in the 10th position
    cols = list(df.columns)
    cols.insert(10, cols.pop(cols.index('route_intersection_origin')))
    df = df[cols]
    
    # Remove the original column
    df.drop(columns=['route_intersection_0'], inplace=True)
    
    return df

In [17]:
df = update_route_intersection(df)

In [18]:
df.to_csv("justlooking_subset_v5.csv", index=False)